# Ranking Wikipedia's pages using PageRank algorithm and wikispeedias results

In the data set called `paths_finished`.tsv are stored an enormous records of successfully finished wikispeedias paths.

All thoses paths taken as a unit could be represented as a graph which each node being a wikipedia page. We will try to build up the transition matrix out of this data set and use the *Vanilla Page Rank* algorithm to rank those wikipedias pages from the most to the least *important*. 


## Import the libraries

In [ ]:
import numpy as np

from itertools import pairwise
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, col
from pyspark.sql.types import StringType, ArrayType, StructField, StructType, IntegerType
from urllib.parse import unquote

## Utils

In [ ]:
def rewrite_path(path):
    """
    This function rewrites path to remove the back-clicks and instead put the right wikipedia pages the uses is back on.
    This will help building the adjacency matrix afterwards.

    Parameters:
    -----------
    path : Row
        Wikispeedias path.

    col : string
        Column name where the path must be extracted from.

    Returns:
    --------
    Row
        Rewritten path
    """

    splitted_path = path.split(";")
    rewritten_path = []
    step_backs = 0  # Indicate how far we should go back to get onto the right page after a series of back-clicks
    idx = 0  # Keep track of the index inside the rewritten path to help us retrieve the previous page after a series of back-clicks
    for page in splitted_path:
        if page != "<":
            if step_backs != 0:
                previous_page = rewritten_path[
                    idx - step_backs - 1
                ]  # -1 here since we are a step ahead the encounter of the most recent back-click character
                rewritten_path.append(previous_page)  # Add the page we landed on after the series of back-clicks
                rewritten_path.append(page)  # Add the current page that marks the end of the back-clicks series
                idx += 2  # Increment by 2 since we are adding two elements to the rewritten path
                step_backs = 0  # Reset the number of step back

            else:
                rewritten_path.append(page)
                idx += 1

        else:
            step_backs += 1

    rewritten_path = [unquote(path) for path in rewritten_path]  # Decode the pages' namesr

    return ";".join(rewritten_path)

In [ ]:
def outgoing_links(path, pages_index):
    """
    Converts a path into a list of edges (source, target) for the outgoing links representation.

    Parameters
    ----------
    path : str
        Rewritten path.

    pages_index : dict
        A dictionary mapping page names to unique indices.

    Returns:
    --------
    list
        List of (source, target) pairs representing edges.
    """
    # Split the path and create pairs
    splitted_path = path.split(";")
    edges = []
    for current_page, next_page in pairwise(splitted_path):
        idx_current_page = pages_index[current_page]
        idx_next_page = pages_index[next_page]
        edges.append((idx_current_page, idx_next_page))
    return edges

In [ ]:
def stochastic_matrix(adjacency_matrix):
    """
    This function creates the transition matrix from the adjacency matrix.

    Parameters
    ----------
    adjacency_matrix : numpy.ndarray of shape (n_pages, n_pages)
        The adjacency matrix related to the history of the wikispeedias games.

    Returns:
    --------
    numpy.ndarray of shape (n_pages, n_pages)
        The transition matrix.
    """

    columns_sum = np.sum(adjacency_matrix, axis=0)
    transition_matrix = adjacency_matrix / columns_sum

    return transition_matrix

In [ ]:
def page_rank(transition_matrix, beta, epsilon, max_iter=1000):
    """
    This function runs the Vanilla Page Rank algorithm including the concept of teleportation to avoid falling into the so-called 'Spider-Trap'.

    Parameters
    ----------
    transition_matrix : numpy.ndarray of shape (n_pages, n_pages)
        The transition matrix.
    beta : float
        The weight put on relying only on the transition matrix.
    epsilon : float
        The convergence threshold.
    max_iter : int, optional
        The maximum number of iterations to prevent infinite loops (default is 1000).

    Returns:
    --------
    numpy.ndarray of shape (n_pages, 1)
        The importance vector
    """

    # Initialize the teleportation vector
    n_pages = transition_matrix.shape[0]
    c = np.ones((n_pages, 1), dtype=float) / n_pages

    # Initialize importance vector with equal importance for each page
    importance_vector = np.copy(c)

    # Iteratively update importance vector until convergence
    for _ in range(max_iter):
        next_importance_vector = beta * (transition_matrix @ importance_vector) + (1 - beta) * c
        if np.linalg.norm(importance_vector - next_importance_vector) < epsilon:
            print("OK")
            break
        importance_vector = next_importance_vector

    return importance_vector

## Reading/Cleaning the data set

In [97]:
# Initiate the PySpark Session
spark = SparkSession.builder.appName("PageRankApp").config("spark.python.worker.serializer", "cloudpickle").getOrCreate()

In [98]:
# Read the file without headers
paths_rdd = spark.read.csv("../data/paths_finished.tsv", sep="\t", header=False).rdd

# Separate the header (line 16) from the rest
header = paths_rdd.zipWithIndex().filter(lambda x: x[1] == 15).map(lambda x: x[0]).collect()[0]
data = paths_rdd.zipWithIndex().filter(lambda x: x[1] > 15).map(lambda x: x[0])

# Convert the RDD to a DataFrame using the header
paths_df = data.toDF(header)

paths_df_reduced = paths_df.select(["path"])

## Rewritting the paths

In [100]:
# Save the function as udf
rewrite_path_udf = udf(rewrite_path, StringType())

# Apply the udf to create a data frame with the rewritten paths
rewritten_path_df = paths_df_reduced.select(rewrite_path_udf(paths_df_reduced["path"]).alias("rewritten_path"))

In [101]:
# Collect all values into a single list
all_paths = rewritten_path_df.select("rewritten_path").rdd.flatMap(lambda row: row["rewritten_path"].split(";")).collect()

# Convert to a set to remove duplicates
unique_pages = set(all_paths)

# Sort alphabetically
sorted_unique_pages = sorted(unique_pages)

## Create the adjacency and transition matrices

In [102]:
pages_index = {}  # This will associate an index to each pages, it will facilitate the updating of pages_associations

# Initialize the pages index
pages_index = {key: idx for idx, key in enumerate(sorted_unique_pages)}

# Define schema for adjacency list (edge list)
schema = StructType([StructField("source", IntegerType(), False), StructField("target", IntegerType(), False)])

In [ ]:
# Register a UDF that creates adjacency list for each path
outgoing_links_udf = udf(lambda path: outgoing_links(path, pages_index), ArrayType(ArrayType(IntegerType())))

# Apply UDF and explode the resulting adjacency list to create an edge DataFrame
adjacency_list_df = rewritten_path_df.select(explode(outgoing_links_udf("rewritten_path")).alias("edge"))
adjacency_list_df = adjacency_list_df.select(col("edge").getItem(0).alias("source"), col("edge").getItem(1).alias("target"))

# Remove duplicates if the graph is undirected, or if you don’t want multiple edges
adjacency_list_df = adjacency_list_df.distinct()

In [105]:
# Initialize empty matrix
adjacency_matrix = np.zeros((len(pages_index), len(pages_index)), dtype=int)

# Fill the adjacency matrix
edges = adjacency_list_df.collect()
for row in edges:
    adjacency_matrix[row["target"], row["source"]] = 1

# For the pages where there is no outgoing links then add a link between the node and itself
pages_idx_no_outgoing_links = np.where(np.sum(adjacency_matrix, axis=0) == 0)

for page_idx in pages_idx_no_outgoing_links:
    adjacency_matrix[page_idx, page_idx] = 1

## Create the transition matrix

In [107]:
transition_matrix = stochastic_matrix(adjacency_matrix)

## Apply the *Vanilla Page Rank* algorithm

In [118]:
importance_vector = page_rank(transition_matrix, 0.8, 1e-5)

OK


In [119]:
# Let's create a dictionary that will attached to each index a page
idx_to_page = {v: k for k, v in pages_index.items()}

In [120]:
# Retrieve the top 10
rankings = np.argsort(importance_vector, axis=0)[::-1]
top_10_idx = rankings[:10]
top_10 = [idx_to_page[elem] for elem in top_10_idx.flatten()]

In [121]:
# Print out the top 10
top_10

['United_States',
 'Europe',
 'United_Kingdom',
 'England',
 'World_War_II',
 'France',
 'Africa',
 'Germany',
 'English_language',
 'India']